# Script Purpose

Calculate the frequency of co-mentions of brands.

For example, if Honda and Toyota are mentioned in the same post, then the co-mention frequency of Honda and Toyota increases by 1.

In [1]:
import pandas as pd
import itertools

In [26]:
metioned_brands = pd.read_csv('mentioned_brands.csv')
metioned_brands

,brand,freq
0,think,1927
1,ford,472
2,local,443
3,bmw,369
4,toyota,279
...,...,...
136,hudson,1
137,hummer,1
138,kaiser,1
139,nash,1


For simplicity, only select brands that were mentioned over 20 times.

We cannot gain much insight if the brand were only mentioned very few time.

In [28]:
metioned_brands = metioned_brands[metioned_brands['freq'] >= 20]
metioned_brands

,brand,freq
0,think,1927
1,ford,472
2,local,443
3,bmw,369
4,toyota,279
5,jeep,274
6,white,243
7,honda,240
8,seat,199
9,audi,190


In [29]:
# Extract the metioned brand names
brands = metioned_brands['brand'].tolist()
brands[:5]

['think', 'ford', 'local', 'bmw', 'toyota']

Remove words that are commonly used in the sentence but not refer to the brand because the manufacturers have already been defuncted.

In [32]:
unwanted = ['think','local','white','seat','mini','standard','bet','buddy','man','smart','national','cutting']
brands = [ele for ele in brands if ele not in unwanted]

len(brands)

35

In [4]:
comments = pd.read_csv('forum_comments_2022.csv')
comments.head()

,date,user_id,message
0,2022-01-01 03:53:00,benjaminh,\nAlthough the acceleration times in most car ...
1,2022-01-01 03:57:00,tjc78,\n\n@stickguy said:\nXM only comes on it if yo...
2,2022-01-01 04:08:00,tjc78,\n\n@qbrozen said:\nI don’t have the room for ...
3,2022-01-01 04:48:00,graphicguy,\nCongrats @stickguy ……very cool!
4,2022-01-01 06:03:00,au1994,\nHappy New Year all!Congrats @stickguy. I rea...


In [6]:
messages = comments['message'].tolist()
messages[:5]

['\nAlthough the acceleration times in most car magazines are somewhat bogus, they are still useful for comparison. Anyway, Car and Driver has just tested a Maverick 2.0 awd, and they got a 0-60 time of 5.9, which is not that far from a base BMW 330i at 5.6. Anyway, for the money the Maverick is a fast vehicle.https://www.caranddriver.com/reviews/a38516737/2022-ford-maverick-xlt-fx4-by-the-numbers/stickguy: Did you end up going for the 0% financing? If so, those payments of c. 750 a month are steep, but the equity will obviously build up very quickly. Even right now you could probably sell it for several thousand more than what you bought it for. The latest I heard on the chip shortage is that it will only slowly abate in 2022. According to one article I saw, things aren\'t likely to truly get back to "normal," whatever that is, probably until the second half of 2023. If true, this will likely mean that there won\'t be a sudden collapse of car prices. With all the lost production and p

In [33]:
# Create a dataframe of size messages to store all the brands mentioned in each message
comention = pd.DataFrame(index=range(len(messages)),columns=['brands_mentioned'])
comention['brands_mentioned'] = pd.np.empty((len(comention), 0)).tolist()
comention.head()

C:\Users\shans\AppData\Local\Temp\ipykernel_24468\3371712461.py:3: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  comention['brands_mentioned'] = pd.np.empty((len(comention), 0)).tolist()


,brands_mentioned
0,[]
1,[]
2,[]
3,[]
4,[]


In [34]:
# For each message
for i in range(len(messages)):
    # For every word in the message
    text = set(messages[i].lower().replace('\n','').split(" "))
    #print(text)
    for w in text:
        if w in brands and w not in comention['brands_mentioned'][i]:
            comention['brands_mentioned'][i].append(w)

comention

,brands_mentioned
0,[bmw]
1,[]
2,[]
3,[]
4,[]
...,...
13498,[]
13499,[]
13500,[]
13501,[]


In [40]:
comention_dict = {}

for i in range(len(comention)):
    brands_in_msg = comention['brands_mentioned'][i]
    brands_in_msg.sort()

    keys = list(itertools.combinations(brands_in_msg,2))
    for k in keys:
        if k in comention_dict:
            comention_dict[k] += 1
        else:
            comention_dict[k] = 1

sorted(comention_dict.items(), key=lambda x:x[1], reverse=True)[:5]

[(('honda', 'toyota'), 50),
 (('acura', 'honda'), 33),
 (('bmw', 'toyota'), 26),
 (('ford', 'toyota'), 26),
 (('bmw', 'jeep'), 24)]

The top 3 brand pairs which get mentioned together the most are:
- Honda and Toyota
- Acura and Honda
- BMW and Toyota / Ford and Toyota